In [3]:
%cd /workspace/src
%load_ext autoreload
%autoreload 2

/workspace/src


# Tutorial: How to use the Flashback-GPT model?

This notebook will walk through how to use the Flashback-GPT model and show some helpful utilities to easily work with it.

In [4]:
import json
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from format import format_thread, format_header, format_thread_post, format_post_item, parse_thread
from flashback_gpt import generate_post

## Load model and tokenizer

Before we start, the model archive needs to be unpacked in the `data/` dir at the root of the repository. Once we got the model and tokenizer files in place, we can load it up:

In [5]:
tokenizer = GPT2TokenizerFast.from_pretrained("../data/model")
model = GPT2LMHeadModel.from_pretrained("../data/model")

## Raw generation from the language model

Perhaps the model is best explained by an example: What happens if we let the language model loose and ask it to generate a continuation from a single-word prompt "Samhälle"?

In [9]:
input_ids = tokenizer.encode("Samhälle", return_tensors="pt")
output = model.generate(input_ids, 
                        max_length=300,
                        top_k=50,
                        do_sample=True,
                        no_repeat_ngram_size=3)
output_text = tokenizer.decode(output[0])
print(output_text)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Samhälle > Företagsskvaller
Varning för Telge Energi

[Användare anonymiserad]:
Citat: [Användare anonymiserad]
	Du bor sjalv betala for att ha ett fungerande elsystem eller menar du att du vill betala for el som du inte paverkar?
Jag är ingen människa, utan en människa. Jag får väl försöka hitta ett elsystem som tillfredställer mina behov.
Citat i trådstarten så ser elcentralen ut så här, och enligt det jag läst, och gjort så bör jag ta tag i detta nu.
För övrigt bor jag inte alls i Stockholm. Jag bor i en förort och lever ett självständigt liv.

[Användare anonymiserad]:
Telge energi är skit tyvärr, jag känner ett flertal andra som jobbar där. Även när man ringer till dom så svarar dom aldrig utan bara babblar på med reklam. Men det är inte alla som ringer eller besöker dem som klagar direkt..

[Användare anonymiserad]:
Jag var väldigt skeptisk till de där energiåterförsäljarna, sedan dess har jag inte orkat gå dit. Har köpt så mycket via telefon och liknande att jag inte längre ids 

We can see that the generated output looks quite structured, which is not strange since the training data was structured in this particular way. What we get is a generated Flashback forum thread, formatted as raw text.

The first line is the Flashback forum (subforums separated by `>`).

The second line is the thread title.

What follows then are thread posts separated by double newline. Each post starts with the username and a colon, followed by one or several lines of post body. 

A post can also _quote_ a previous post by a certain user, and in such case the quoted post will be indented with a tab.

## Structured representation of a thread

This repo contains a function to parse the generated text into a structured format (`parse_thread`).

In [10]:
parsed_thread = parse_thread(output_text)
print(json.dumps(parsed_thread, indent=2, ensure_ascii=False))

{
  "forumTitle": "Samhälle > Företagsskvaller",
  "threadTitle": "Varning för Telge Energi",
  "posts": [
    {
      "username": "[Användare anonymiserad]",
      "post": [
        {
          "type": "quote",
          "username": "[Användare anonymiserad]",
          "post": [
            {
              "type": "text",
              "text": "Du bor sjalv betala for att ha ett fungerande elsystem eller menar du att du vill betala for el som du inte paverkar?"
            }
          ]
        },
        {
          "type": "text",
          "text": "Jag är ingen människa, utan en människa. Jag får väl försöka hitta ett elsystem som tillfredställer mina behov."
        },
        {
          "type": "text",
          "text": "Citat i trådstarten så ser elcentralen ut så här, och enligt det jag läst, och gjort så bör jag ta tag i detta nu."
        },
        {
          "type": "text",
          "text": "För övrigt bor jag inte alls i Stockholm. Jag bor i en förort och lever ett sjä

The repo also contains a function (`format_thread`) to go from the structured back to the textual representation:

In [11]:
textual_thread = format_thread(parsed_thread)[0]
print(textual_thread)

Samhälle > Företagsskvaller
Varning för Telge Energi

[Användare anonymiserad]:
Citat: Fantomsmaerta
	Du bor sjalv betala for att ha ett fungerande elsystem eller menar du att du vill betala for el som du inte paverkar?
Jag är ingen människa, utan en människa. Jag får väl försöka hitta ett elsystem som tillfredställer mina behov.
Citat i trådstarten så ser elcentralen ut så här, och enligt det jag läst, och gjort så bör jag ta tag i detta nu.
För övrigt bor jag inte alls i Stockholm. Jag bor i en förort och lever ett självständigt liv.

[Användare anonymiserad]:
Telge energi är skit tyvärr, jag känner ett flertal andra som jobbar där. Även när man ringer till dom så svarar dom aldrig utan bara babblar på med reklam. Men det är inte alla som ringer eller besöker dem som klagar direkt..

[Användare anonymiserad]:
Jag var väldigt skeptisk till de där energiåterförsäljarna, sedan dess har jag inte orkat gå dit. Har köpt så mycket via telefon och liknande att jag inte längre ids räkna på va

## Generating from the structured representation

By using both the `format_thread` and `parse_thread` functions, the `generate_post` function can take a structured thread object and generate a post _conditioned on the thread_. Internally, it first formats the thread and generates until the post terminates (=double newline), and finally parses everything back to the structured representation.

Let's look at an example. Here we want to generate a post conditioned on the following thread:

In [ ]:
thread = {
    "forumTitle": "Samhälle > Juridik",
    "threadTitle": "Min granne väsnas",
    "posts": [{
        "username": "arggranne",
        "post": [
            {"text": "Hej! Min granne har fest hela tiden och väsnas något oerhört.", "type": "text"},
            {"text": "Kan jag anmäla honom för något?", "type": "text"}
        ]
    }]
}

That means the language model will be prompted with the following:

In [16]:
print(format_thread(thread)[0])

Samhälle > Juridik
Min granne väsnas

arggranne:
Hej! Min granne har fest hela tiden och väsnas något oerhört.
Kan jag anmäla honom för något?




So let's try to generate a continuation to this thread using the `generate_post` function:

In [17]:
post, generated_text = generate_post(model, tokenizer, thread)

The function returns two things: `post` is the structured representation of the generated post (dict), and `generated_text` is the full text including the prompt (str)

In [19]:
print(generated_text)

Samhälle > Juridik
Min granne väsnas

arggranne:
Hej! Min granne har fest hela tiden och väsnas något oerhört.
Kan jag anmäla honom för något?

[Användare anonymiserad]:
Ja, det kan du.




In [20]:
print(json.dumps(post, indent=2, ensure_ascii=False))

{
  "username": "[Användare anonymiserad]",
  "post": [
    {
      "type": "text",
      "text": "Ja, det kan du."
    }
  ]
}


Sometimes the model generates fairly short an boring responses. This can be often be fixed by punishing shorter posts with the `length_penalty` argument (higher number promotes longer posts)

In [24]:
post, generated_text = generate_post(model, tokenizer, thread,
                                     length_penalty=5)
print(generated_text)

Samhälle > Juridik
Min granne väsnas

arggranne:
Hej! Min granne har fest hela tiden och väsnas något oerhört.
Kan jag anmäla honom för något?

[Användare anonymiserad]:
Nej det kan du inte.
Det är inte olagligt att störa sina grannar, det är däremot olagligt att göra det.
Om du vill göra det får du göra det själv.
Du får dock inte störa dina grannar mer än vad som är nödvändigt för att det ska räknas som störande av den allmänna ordningen.
Att du inte får störa dina grannars festande är en annan sak.
Edit: Det är inte heller olagligt att spela hög musik på allmän plats, så länge det inte är störande för någon annan.
http://www.notisum.se/rnp/sls/lag/19700994.htm#K9P4S1
Där kan du läsa lite om vad som gäller.
EDIT2: Du kan inte göra något åt det, det enda du kan göra är att prata med hyresvärden och säga att du inte vill att han ska störa dina fester. Det är inget olagligt med det, men det är heller inget som hyresvärden kan hjälpa dig med.
edit3: Om det är en hyresrätt kan du vända di

The `generate_post` function internally uses the `beam_sample()` method provided by `transformers` to decode from the model. This is also what was used in the paper as it trades short/booring responses against long but incoherent somewhat well. This could probably be optimized further!